In [1]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from time import gmtime, strftime
import mlflow
import pandas as pd
import numpy as np
import warnings
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import ticker
from statsmodels.tsa.stattools import adfuller, acf, pacf
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# suppressing warning messages

warnings.filterwarnings('ignore')

In [ ]:
sys.path.insert(1, '../scripts')

from data_viz import Data_Viz
from data_cleaning import DataCleaner
from data_transformation import DataTransformer

DV = Data_Viz("../logs/deep_model_notebook.log")
DC = DataCleaner("../logs/deep_model_notebook.log")
DT = DataTransformer("../logs/deep_model_notebook.log")

In [ ]:
# importing the data

train = pd.read_csv("../data/train_store.csv")

In [ ]:
# checking available variables
train = train.sort_values(by="Date")
train["Date"] = pd.to_datetime(train["Date"])

print("rows and columsn: ", train.shape)
DV.summ_columns(train)

In [ ]:
# Sales per Day

tsa_df_days = train[["Date", "Sales"]].groupby(["Date"]).agg({"Sales":"mean"})

print("rows and columns: ", tsa_df_days.shape)
print(DV.summ_columns(tsa_df_days))
tsa_df_days.head()

In [ ]:
DV.plot_line(tsa_df_days, "Date", "Sales", [15, 5], "sales per day", "sales_day.png")

In [ ]:
# creating week based data
tsa_df_weeks = train[["WeekOfYear", "Sales"]].groupby(["WeekOfYear"]).agg({"Sales": "mean"})
print("rows and columns: ", tsa_df_weeks.shape)
print(DV.summ_columns(tsa_df_weeks))
tsa_df_weeks.head()

In [ ]:
DV.plot_line(tsa_df_weeks, "WeekOfYear", "Sales", [15, 5], "sales per weeks", "sales_weeks.png")

**Observation**
- The data seems like it has somewhat constant mean and SD.
- There doesn't seem to be any seasonality as well.
- as a result it is probably a stationary data
- but let's verify that

## Checking for Stationarity

In [ ]:
# using variance and mean
def check_stationary_mv(df, title):
    print("\n", title)
    X = df[1:].values
    split = round(len(X) / 2)
    X1, X2 = X[0:split], X[split:]
    mean1, mean2 = X1.mean(), X2.mean()
    var1, var2 = X1.var(), X2.var()
    print('mean1= %f, mean2= %f' % (mean1, mean2))
    print('variance1= %f, variance2= %f' % (var1, var2))

check_stationary_mv(tsa_df_days, "sales per days")
check_stationary_mv(tsa_df_weeks, "sales per weeks")

In [ ]:
# using The Augmented Dickey-Fuller test
# the null hypothesis is that the series is non-stationary
# if p-value is < 0.05 we will reject the null hypothesis

def check_stationary(df, col, title):
    print("\n",title )
    adfResult = adfuller(df[col].values, autolag='AIC')
    print(f'ADF Statistic: {adfResult[0]}')
    print(f'p-value: {adfResult[1]}')
    return adfResult

check_stationary(tsa_df_days, "Sales", "sales per day")
check_stationary(tsa_df_weeks, "Sales", "sales per weeks")
print("")

**Observation**
- the dataset is stationary because ADF is big negative and p-value is below 0.005

## checking autocorrelation

In [ ]:
def corrPlots(array: np.array, prefix: str):
    plt.figure(figsize=(30, 5))
    plt.title(f"{prefix}  Autocorrelations of Sales")
    plt.bar(range(len(array)), array)
    plt.grid(True)
    plt.show()

In [ ]:
# plotting autocorrelation graph for sales per day
acf_days = acf(tsa_df_days.Sales.values, fft=True, nlags=941)
acfNp = np.array(acf_days)
pacf_days = pacf(tsa_df_days.Sales.values, nlags=200)
pacfNp = np.array(pacf_days)

corrPlots(acfNp, '')
corrPlots(pacfNp, "Paritial")

In [ ]:
# plotting autocorrelation graph for sales per weeks
plt.rc("figure", figsize=(10,6))
pd.plotting.autocorrelation_plot(tsa_df_weeks['Sales'])

# scaling the data

In [ ]:
# scaling the data from -1 go 1
scaler = MinMaxScaler()
scaler.fit(tsa_df_days.Sales.values.reshape([-1, 1]))
SalesScaled = scaler.transform(tsa_df_days.Sales.values.reshape(-1, 1))
tsa_df_days['SalesScaled'] = SalesScaled
tsa_df_days.tail(10)

## Training and Validation set separation

In [ ]:
SIZE = len(tsa_df_days.SalesScaled)
WINDOW_SIZE = 48
BATCH_SIZE= SIZE-WINDOW_SIZE*2
EPOCHS = 200

In [ ]:
DateTrain = tsa_df_days.index.values[0:BATCH_SIZE]
DateValid = tsa_df_days.index.values[BATCH_SIZE:]
XTrain = tsa_df_days.SalesScaled.values[0:BATCH_SIZE].astype('float32')
XValid = tsa_df_days.SalesScaled.values[BATCH_SIZE:].astype('float32')

# Obtain shapes for vectors of size (,1) for dates series

DateTrain = np.reshape(DateTrain, (-1, 1))
DateValid = np.reshape(DateValid, (-1, 1))

print("Shape of the training set date series: ", DateTrain.shape)
print("Shape of the validation set date series: ", DateValid.shape)
print()
print("Shape of the training set logarithm of sales series: ", XTrain.shape)
print("Shape of the validation set logarithm of sales series in a stateless LSTM: ", XValid.shape)

In [ ]:
tf.random.set_seed(1234)
# add extra dimension
series = tf.expand_dims(XTrain, axis=-1)
series.shape

In [ ]:
# create tensor from each individual element
dataset = tf.data.Dataset.from_tensor_slices(series)
dataset

In [ ]:
# takes a window_size + 1 chunk from the slices
dataset = dataset.window(WINDOW_SIZE + 1, shift=1, drop_remainder=True) 

In [ ]:
# Example of Window
datasetEx = tf.data.Dataset.from_tensor_slices(tf.range(10))
datasetEx = datasetEx.window(5, shift=1, drop_remainder=True)
for window in datasetEx:
    print([elem.numpy() for elem in window])

In [ ]:
dataset = dataset.flat_map(lambda window: window.batch(WINDOW_SIZE + 1))

In [ ]:
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))

In [ ]:
dataset = dataset.batch(BATCH_SIZE).prefetch(1)

In [ ]:
def windowed_dataset(series, window_size=WINDOW_SIZE, batch_size=BATCH_SIZE): 
  series = tf.expand_dims(series, axis=-1)
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True) 
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [ ]:
DatasetTrain = windowed_dataset(XTrain)
DatasetVal = windowed_dataset(XValid)

In [ ]:
model = Sequential()
model.add(LSTM(8, input_shape=[None, 1], return_sequences=True))
model.add(LSTM(4, input_shape=[None, 1]))
model.add(Dense(1))
model.compile(loss="huber_loss", optimizer='adam')

In [ ]:
model.summary()

In [ ]:
mlflow.set_experiment("LSTM modeling experiment")
mlflow.tensorflow.autolog()
history = model.fit(DatasetTrain, epochs=EPOCHS, validation_data=DatasetVal, verbose=1)

In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'], label="loss")
plt.plot(history.history['val_loss'], label="val_loss")
plt.legend()
plt.show()

## Assess Accuracy of Forecast

In [ ]:
def model_forecast(model, series, window_size):
  ds = tf.data.Dataset.from_tensor_slices(series)
  ds = ds.window(window_size, shift=1, drop_remainder=True) 
  ds = ds.flat_map(lambda w: w.batch(window_size))
  ds = ds.batch(SIZE).prefetch(1)
  forecast = model.predict(ds)
  return forecast

In [ ]:
Forecast = model_forecast(model, houseSales.HouseSalesScaled.values[:, np.newaxis], WINDOW_SIZE)
Results = Forecast[BATCH_SIZE-WINDOW_SIZE:-1]
Results1 = scaler.inverse_transform(Results.reshape(-1,1))
XValid1 = scaler.inverse_transform(XValid.reshape(-1,1))

In [ ]:
plt.figure(figsize=(30, 8))
plt.title("LSTM Model Forecast Compared to Validation Data")
plt.plot(DateValid.astype('datetime64'), Results1, label='Forecast series')
plt.plot(DateValid.astype('datetime64'), np.reshape(XValid1, (2*WINDOW_SIZE, 1)), label='Validation series')
plt.xlabel('Date')
plt.ylabel('Thousands of Units')
plt.xticks(DateValid.astype('datetime64')[:,-1], rotation = 90) 
plt.legend(loc="upper right")

MAE = tf.keras.metrics.mean_absolute_error(XValid1[:,-1], Results[:,-1]).numpy()
RMSE = np.sqrt(tf.keras.metrics.mean_squared_error(XValid1[:,-1], Results[:,-1]).numpy())

textstr = "MAE = " + "{:.3f}".format(MAE) + "  RMSE = " + "{:.3f}".format(RMSE)

# place a text box in upper left in axes coords
plt.annotate(textstr, xy=(0.87, 0.05), xycoords='axes fraction')
plt.grid(True)

plt.show()

In [ ]:
print(textstr)

In [ ]:
time = strftime("%Y-%m-%d-%H:%M:%S", gmtime())
model.save(f'../models/LSTM_sales {time}.pkl')